In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from dateutil.parser import parse
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from tqdm import tqdm

In [8]:
test_trans = pd.read_csv('./data/transactions_test.csv', parse_dates=["TX_TS"])
merchants_full = pd.read_csv('./data/merchants.csv')

terminals_full = pd.read_csv('./data/terminals.csv')

customers_full = pd.read_csv('./data/customers.csv')

In [13]:
test_trans = pd.merge(test_trans,merchants_full,on='MERCHANT_ID')

In [14]:
test_trans = pd.merge(test_trans,terminals_full,on='TERMINAL_ID')

In [15]:
test_trans = pd.merge(test_trans,customers_full,on='CUSTOMER_ID')

In [16]:
useless_columns = ['MERCHANT_ID','TERMINAL_ID','CUSTOMER_ID','CARD_EXPIRY_DATE','TX_ID','CARD_DATA','LEGAL_NAME','FOUNDATION_DATE','ACTIVE_FROM','TRADING_FROM']
test_trans.drop(useless_columns,axis=1,inplace=True)

In [17]:
test_trans['ANNUAL_TURNOVER_CASH'] = test_trans['ANNUAL_TURNOVER'] - test_trans['ANNUAL_TURNOVER_CARD']

In [18]:
bins = [0, 6, 12, 18, 24]
labels = ['00:00-05:59', '06:00-11:59', '12:00-17:59', '18:00-23:59']
test_trans['Time_of_day'] = pd.cut(test_trans.TX_TS.dt.hour, bins, labels=labels, right=False)

In [19]:
bins = np.arange(12)
test_trans['Month'] = test_trans['TX_TS'].apply(lambda x: x.month)

In [20]:
test_trans['Day_of_week'] = test_trans['TX_TS'].apply(lambda x: x.weekday())

In [21]:
test_trans.drop(['TX_TS'],axis=1,inplace=True)

In [22]:
test_trans.loc[test_trans['IS_RECURRING_TRANSACTION'] == 'True',['IS_RECURRING_TRANSACTION']] = 1
test_trans.loc[test_trans['IS_RECURRING_TRANSACTION'] == True,['IS_RECURRING_TRANSACTION']] = 1
test_trans.loc[test_trans['IS_RECURRING_TRANSACTION'] == 'False',['IS_RECURRING_TRANSACTION']] = 0
test_trans.loc[test_trans['IS_RECURRING_TRANSACTION'] == False,['IS_RECURRING_TRANSACTION']] = 0

test_trans['IS_RECURRING_TRANSACTION'] = test_trans['IS_RECURRING_TRANSACTION'].apply(pd.to_numeric)

In [24]:
one_hot_encoded_col = ['CARD_BRAND','TRANSACTION_TYPE','TRANSACTION_STATUS','FAILURE_CODE','FAILURE_REASON','TRANSACTION_CURRENCY','CARD_COUNTRY_CODE','ACQUIRER_ID','CARDHOLDER_AUTH_METHOD','BUSINESS_TYPE','OUTLET_TYPE','Time_of_day']
test_trans = pd.get_dummies(test_trans, columns = one_hot_encoded_col)

In [26]:
location = 'test_features'
test_trans.to_csv(F'./data/{location}/test_trans.csv',index=False)